In [1]:
import numpy as np
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

import abus_classification

## Prof for work

$$\alpha : \angle POQ \ \beta : \angle QOX$$

$$\bar{OP} = R = \sqrt{x_P^2+y_P^2+z_P^2}$$

$$\bar{OQ} = R^{\prime} = R\times COS(\alpha)$$

$$x_p = R^{\prime} \times COS(\beta) = R \times COS(\alpha) \times COS(\beta)$$
$$y_p = R \times SIN(\alpha)$$
$$z_p = R^{\prime} \times SIN(\beta) = R\times COS(\alpha) \times SIN(\beta)$$

Then if we device the vector $\vec{OP}$ by its length $R$ we will get the aigen vector of the line $\vec{OP}$ as follows: